In [ ]:
!pip install sentencepiece
!pip install transformers

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import pickle
from tqdm import tqdm

!pip install deepcut
import deepcut

In [ ]:
def read_txt(path_file):
    txt_list = []
    txtf = open(path_file, 'r', encoding="utf-8")
    line_list = txtf.readlines()
    for line in line_list:
        txt = line
        txt = txt.replace("\n", "")
        txt_list.append(txt)
    return txt_list

def merge_space_list(txt_list):
  txt_merge_list = []
  for i in range(len(txt_list)):
    txt_merge_list.append(txt_list[i].replace(" ", ""))
  return txt_merge_list

In [ ]:
txt_en_news_list = read_txt(os.path.join("val", "dev.en.txt"))
txt_th_news_list = read_txt(os.path.join("val", "dev.th.txt"))
txt_en_conv_list = read_txt(os.path.join("val", "dev2.en.txt"))
txt_th_conv_list = read_txt(os.path.join("val", "dev2.th.txt"))

txt_en_list = txt_en_conv_list + txt_en_news_list
txt_th_list = txt_th_conv_list + txt_th_news_list

In [ ]:
def post_process_en(txt):
  return txt.strip().split()
def post_process_th(txt):
  return txt.strip()
tok_th_list = []
for i in range(len(txt_th_list)):
  tok_th_list.append(deepcut.tokenize(txt_th_list[i]))

In [ ]:
import nltk.translate.bleu_score as bleu_score
def get_blue_score_th2en(pred_list, txt_en_list, show_res=False):
  score_list = []
  for i in range(len(pred_list)):
    pred = pred_list[i]
    ref = txt_en_list[i].split()
    score = bleu_score.sentence_bleu([ref], pred)
    score_list.append(score)
    if show_res:
      print(pred)
      print(ref)
      print(score)
  score_avg = np.mean(np.asarray(score_list))
  return score_avg

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
path_model = "Helsinki-NLP/opus-mt-th-en"

tokenizer = AutoTokenizer.from_pretrained(path_model)
model = AutoModelForSeq2SeqLM.from_pretrained(path_model)
model = model.cuda()

In [ ]:
pred_list = []
for i in tqdm(range(len(txt_th_list))):
  input_feat = tokenizer(txt_th_list[i], return_tensors="pt")
  output_list = model.generate(input_ids=input_feat['input_ids'].cuda(), num_beams=1, num_return_sequences=1)
  output_list = tokenizer.decode(output_list[0], skip_special_tokens=True)
  pred_list.append(post_process_en(output_list))

100%|██████████| 3018/3018 [04:47<00:00, 10.50it/s]


In [ ]:
get_blue_score_th2en(pred_list, txt_en_list)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4036413035599507